In [66]:
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [67]:
# Import the data
train_dataset = pd.read_csv('/kaggle/input/titanic/train.csv')
test_dataset = pd.read_csv('/kaggle/input/titanic/test.csv')

In [68]:
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [69]:
test_dataset.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [70]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [71]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [72]:
train_dataset.isnull().sum().sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [73]:
test_dataset.isnull().sum().sort_values(ascending=False)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Embarked         0
dtype: int64

In [74]:
# Data cleaning

# Drop irrelevant columns on the train dataset
train_dataset.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Drop irrelevant columns on the train dataset
test_dataset.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [75]:
# Handle missing values

# The train dataset
train_dataset['Age'].fillna(train_dataset['Age'].median(), inplace=True)  # Replace missing age values with the median
train_dataset['Embarked'].fillna(train_dataset['Embarked'].mode()[0], inplace=True)  # Replace missing embarked values with the mode

# The test dataset
test_dataset['Age'].fillna(test_dataset['Age'].median(), inplace=True)  # Replace missing age values with the median
test_dataset['Fare'].fillna(test_dataset['Fare'].mode()[0], inplace=True)  # Replace missing embarked values with the mode

In [76]:
train_dataset.isnull().sum().sort_values(ascending=False)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [77]:
test_dataset.isnull().sum().sort_values(ascending=False)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [78]:
# Feature engineering

def feature_engineering(dataset) :
    
    # Create a new feature 'FamilySize' by combining 'SibSp' and 'Parch'
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    return dataset

feature_engineering(train_dataset)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize
0,1,0,3,male,22.0,1,0,7.2500,S,2
1,2,1,1,female,38.0,1,0,71.2833,C,2
2,3,1,3,female,26.0,0,0,7.9250,S,1
3,4,1,1,female,35.0,1,0,53.1000,S,2
4,5,0,3,male,35.0,0,0,8.0500,S,1
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S,1
887,888,1,1,female,19.0,0,0,30.0000,S,1
888,889,0,3,female,28.0,1,2,23.4500,S,4
889,890,1,1,male,26.0,0,0,30.0000,C,1


In [79]:
feature_engineering(test_dataset)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize
0,892,3,male,34.5,0,0,7.8292,Q,1
1,893,3,female,47.0,1,0,7.0000,S,2
2,894,2,male,62.0,0,0,9.6875,Q,1
3,895,3,male,27.0,0,0,8.6625,S,1
4,896,3,female,22.0,1,1,12.2875,S,3
...,...,...,...,...,...,...,...,...,...
413,1305,3,male,27.0,0,0,8.0500,S,1
414,1306,1,female,39.0,0,0,108.9000,C,1
415,1307,3,male,38.5,0,0,7.2500,S,1
416,1308,3,male,27.0,0,0,8.0500,S,1


In [80]:
# Data pre-processing

def data_preprocessing(dataset) :

    # Categorical Feature Encoding
    dataset = pd.DataFrame(dataset)
    
    # Use One-Hot Encoding for 'Embarked' and 'Sex'
    dataset = pd.get_dummies(dataset, columns=['Embarked', 'Sex'], drop_first=True)
    
    # Display the encoded DataFrame
    return dataset

train_dataset = data_preprocessing(train_dataset)
train_dataset.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,Embarked_Q,Embarked_S,Sex_male
0,1,0,3,22.0,1,0,7.2500,2,False,True,True
1,2,1,1,38.0,1,0,71.2833,2,False,False,False
2,3,1,3,26.0,0,0,7.9250,1,False,True,False
3,4,1,1,35.0,1,0,53.1000,2,False,True,False
4,5,0,3,35.0,0,0,8.0500,1,False,True,True


In [81]:
test_dataset = data_preprocessing(test_dataset)
test_dataset.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,FamilySize,Embarked_Q,Embarked_S,Sex_male
0,892,3,34.5,0,0,7.8292,1,True,False,True
1,893,3,47.0,1,0,7.0000,2,False,True,False
2,894,2,62.0,0,0,9.6875,1,True,False,True
3,895,3,27.0,0,0,8.6625,1,False,True,True
4,896,3,22.0,1,1,12.2875,3,False,True,False


In [82]:
# Extract features (X) and target variable (y)
X_train = train_dataset.drop('Survived', axis=1)  # Features
y_train = train_dataset['Survived']  # Target variable

X_test = test_dataset

In [83]:
X_train.dtypes

PassengerId      int64
Pclass           int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
FamilySize       int64
Embarked_Q        bool
Embarked_S        bool
Sex_male          bool
dtype: object

In [95]:
# Instantiate RandomForest model
model = RandomForestClassifier()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Instantiate GridSearchCV with cross-validation (Stratified K-Fold)
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=StratifiedKFold(n_splits=7, shuffle=True),
    scoring='accuracy',
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Print the best hyperparameters
print("Best Hyperparameters for Random Forest:")
print(best_params)

Fitting 7 folds for each of 81 candidates, totalling 567 fits
Best Hyperparameters for Random Forest:
{'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


In [96]:
model = RandomForestClassifier(
    max_depth=5,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=50
)

# Train the final model on the entire training dataset
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [97]:
print(y_pred)

[0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0
 1 0 1 0 0 1 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 0]


In [98]:
# Create a submission DataFrame
submission = pd.DataFrame({
    'PassengerId': X_test['PassengerId'].astype(int),
    'Survived': y_pred.astype(int)
})

In [99]:
print(submission)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [100]:
# If they don't match, reset the index in submission_df
submission.reset_index(drop=True, inplace=True)

# Now, try saving the CSV again
submission.to_csv('submission.csv', index=False)